# PCA analysis of VAE latent space
Use Bokeh interactive plotting to highlight single compund in the latent space by clicking on the selected data point.

In [48]:
import os
import numpy as np
from vae import BaseVAE
import specvae.dataset as dt
import specvae.utils as utils

In [49]:
# Parameters
dataset = "MoNA"
model_name = "joint_vae_30-15-3-15-30 (12-11-2021_04-42-13)"
model_dir = "d:\\Workspace\\SpecVAE\\.model\\MoNA\\joint_vae_3_15peaks\\joint_vae_30-15-3-15-30 (12-11-2021_04-42-13)"


## Load model

In [50]:
device, cpu = utils.device(use_cuda=True)

GPU device count: 1
Device in use:  cuda:0


In [51]:
print("Load model: %s..." % model_name)
model_path = os.path.join(model_dir, 'model.pth')
model = BaseVAE.load(model_path, device)
model.eval()

Load model: joint_vae_30-15-3-15-30 (12-11-2021_04-42-13)...


JointVAE(
  (encoder_): Sequential(
    (en_lin_1): Linear(in_features=30, out_features=15, bias=True)
    (en_lin_batchnorm_1): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (en_act_1): ReLU()
  )
  (en_mu): Linear(in_features=15, out_features=3, bias=True)
  (en_mu_batchnorm): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (en_log_var): Linear(in_features=15, out_features=3, bias=True)
  (en_log_var_batchnorm): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (decoder): Sequential(
    (de_lin_1): Linear(in_features=3, out_features=15, bias=True)
    (de_lin_batchnorm_1): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (de_act_1): ReLU()
    (de_lin_2): Linear(in_features=15, out_features=30, bias=True)
    (de_act_2): ReLUlimit()
  )
  (loss): JointVAECriterium(
    (gnll): GaussianNLLLoss()
  )
)

In [52]:
model.config

{'layer_config': array([[30, 15,  3],
        [ 3, 15, 30]]),
 'limit': 2500,
 'latent_spec': {'cont': 3},
 'temperature': 0.5,
 'cont_capacity': [0.0, 200.0, 10000, 500.0],
 'disc_capacity': [0.0, 1.0, 10000, 10],
 'dropout': 0.0,
 'transform': Compose(
 ),
 'input_columns': ['spectrum'],
 'types': [torch.float32],
 'dataset': 'MoNA',
 'max_mz': 2500,
 'min_intensity': 0.1,
 'max_num_peaks': 15,
 'normalize_intensity': True,
 'normalize_mass': True,
 'n_samples': -1}

## Load and transform data

In [53]:
if dataset == 'MoNA':
    labels = ['ionization_mode_id', 'collision_energy', 'total_exact_mass', 'precursor_mz', 'instrument_type_id', 'precursor_type_id', 'superclass_id', 'class_id']
    base_path = utils.get_project_path() / '.data' / 'MoNA'
    metadata_path = base_path / 'MoNA_meta.npy'
elif dataset == 'HMDB':
    labels = ['ionization_mode_id', 'collision_energy', 'superclass_id', 'class_id']
    base_path = utils.get_project_path() / '.data' / 'HMDB'
    metadata_path = base_path / 'HMDB_meta.npy'

metadata = None
if os.path.exists(metadata_path):
    metadata = np.load(metadata_path, allow_pickle=True).item()

In [54]:
def load_vis_data(target_column):
    data_path = base_path / 'visualization.csv'
    df = dt.Spectra.open(data_path)
    return df

In [55]:
def preload_data_as_tensor(df):
    columns = model.config['input_columns']
    types = model.config['types']
    data = dt.Spectra.preload_tensor(
        device=device, data_frame=df[columns + ['id']], transform=model.transform, limit=-1, types=types, do_print=False)
    return data

In [56]:
def evaluate_model(df, data):
    print("Encode N=%d instances from %s dataset..." % (data['id'].shape[0], dataset))
    X, ids = data['spectrum'], data['id'] # TODO: handle the case for concatanated input
    Xrecon, z, latent_dist = model.forward_(X)
    print(z.shape)
    data_np = {}
    data_np['X'] = X.data.cpu().numpy()
    data_np['Xrecon'] = Xrecon.data.cpu().numpy()
    data_np['z'] = z.data.cpu().numpy()
    data_np['ids'] = ids
    data_np['ionization_mode_id'] = df['ionization_mode_id'].to_numpy()
    data_np['collision_energy'] = df['collision_energy'].to_numpy()
    # data_np['images'] = df['images'].to_numpy()
    return data_np


## Prepare data for vizualization

In [57]:
import torch
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import UMAP
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='white', context='notebook', rc={'figure.figsize':(24,20)})

import torchvision as tv
import specvae.vae as vae, specvae.utils as utils
import plotly.io as pio
pio.renderers.default='notebook'
import plotly.express as px
import pandas as pd

In [58]:
def get_colors(df, data_np):
    colors = {}
    colors['ionization_mode_id'] = np.array(list(map(lambda x: 'negative' if x==0 else 'positive', data_np['ionization_mode_id'].tolist())))
    colors['collision_energy'] = data_np['collision_energy']
    colors['superclass_id'] = df['superclass'].to_numpy()
    df['class'] = df['class'].fillna('Undefined')
    colors['class_id'] = df['class'].to_numpy()
    if dataset == 'MoNA':
        df['precursor_type'] = df['precursor_type'].fillna('Undefined')
        colors['precursor_type_id'] = df['precursor_type'].to_numpy()
        df['instrument_type'] = df['instrument_type'].fillna('Undefined')
        colors['instrument_type_id'] = df['instrument_type'].to_numpy()
        colors['total_exact_mass'] = df['total_exact_mass'].to_numpy()
        colors['precursor_mz'] = df['precursor_mz'].to_numpy()
    return colors

In [59]:
def compute_pca(data, n=2):
    print("Compute PCA for n_components=%d" % n)
    red = PCA(n)
    rdata = red.fit_transform(data)
    print("\t      explained_variance:", red.explained_variance_)
    print("\texplained_variance_ratio:", red.explained_variance_ratio_)
    return rdata

def compute_tsne(data, n=2):
    print("Compute tSNE for n_components=%d" % n)
    r = TSNE(n)
    rdata = r.fit_transform(data)
    print("TSNE:")
    print("\t      kl_divergence:", r.kl_divergence_)
    return rdata

def plot_data(data, labels=None, label_name=None, plot_components=2, 
        hover_data=None, width=1000, height=1000, title='Visualization of dataset'):
    if plot_components == 2:
        fig = px.scatter(data, x=0, y=1, color=labels, 
            template='plotly_white', hover_data=hover_data,
            title=title, width=width, height=height)
        fig.show()
    elif plot_components == 3:
        fig = px.scatter_3d(data, x=0, y=1, z=2, color=labels, 
            template='plotly_white', hover_data=hover_data, 
            title=title, width=width, height=height)
        fig.update_traces(
            marker=dict(size=3),
            selector=dict(mode='markers'))
        fig.show()

## Bokeh visualization

In [60]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, LinearColorMapper, ColorBar, TapTool
from bokeh.palettes import Spectral10
from bokeh.palettes import RdBu3
from bokeh.models.callbacks import CustomJS
output_notebook()

Loading BokehJS ...

In [64]:
def plot_hover(data, target_column, title=""):
    ds = {
        'x':        data['z'][:, 0],
        'y':        data['z'][:, 1],
        'energy':   data['collision_energy'],
        'mode':     data['ionization_mode_id'],
        'ids':      data['ids'],
        # 'label':    colors[target_column],
        'color':    colors[target_column]
    }
    datasource = ColumnDataSource(ds)
    plot_figure = figure(
        title=title,
        plot_width=1000,
        plot_height=1000)
    plot_figure.add_tools(HoverTool(tooltips="""
    <div>
        <div>
            <span style='font-size: 11px; color: #224499'>ID:</span>
            <span style='font-size: 11px'>@ids</span>
        </div>
        <div>
            <span style='font-size: 11px; color: #224499'>Mode:</span>
            <span style='font-size: 11px'>@mode</span>
        </div>
        <div>
            <span style='font-size: 11px; color: #224499'>Energy:</span>
            <span style='font-size: 11px'>@energy</span>
        </div>
    </div>
    """))
    
    cb_click = CustomJS(args=dict(source=datasource), code="""
        var data = source.data
        var selected = source.selected.indices
        var select_inds = []
        var prop_name = 'ids'
        if(selected.length == 1) {
            var selected_prop = data[prop_name][selected[0]]
            for (var i = 0; i < data['x'].length; ++i) {
                if(data[prop_name][i] == selected_prop) {
                    data['color'][i] = '#ff0000'
                    select_inds.push(i)
                }
            }
        }
        source.selected.indices = select_inds 
        source.change.emit();
    """)
    plot_figure.add_tools(TapTool(callback=cb_click))
    plot_figure.circle(
        'x', 'y',
        source=datasource,
        # color='color',
        # legend_group='label',
        # line_alpha=0.6,
        # fill_alpha=0.6,
        size=6)
#     color_bar = ColorBar(color_mapper=color_mapping, label_standoff=12)
#     plot_figure.add_layout(color_bar, 'right')
    show(plot_figure)

In [62]:
target_column = 'collision_energy'
df = load_vis_data(target_column)
data = preload_data_as_tensor(df)
data_np = evaluate_model(df, data)
colors = get_colors(df, data_np)
n_dim = data_np['z'].shape[1]

Encode N=5832 instances from MoNA dataset...
torch.Size([5832, 3])


In [65]:
plot_hover(data_np, target_column)

## Plot 2D PCA for different labels

In [ ]:
for label in labels:
    df = load_vis_data(label)
    data = preload_data_as_tensor(df)
    data_np = evaluate_model(df, data)
    colors = get_colors(df, data_np)
    n_dim = data_np['z'].shape[1]

    if n_dim > 2:
        pca2_data = compute_pca(data_np['z'], n=2)
        plot_data(pca2_data, colors[label], label, 
            hover_data={
                'ids': data_np['ids'], 
                'mode': data_np['ionization_mode_id'], 
                'energy': data_np['collision_energy']},
            title='PCA analysis of VAE latent space for %s dataset (%s)' % (dataset, label) 
                    if label else 'PCA analysis of VAE latent space for %s dataset' % dataset)
    elif n_dim == 2:
        plot_data(data_np['z'], colors[label], label, 
            hover_data={
                'ids': data_np['ids'], 
                'mode': data_np['ionization_mode_id'], 
                'energy': data_np['collision_energy']},
            title='Direct representation of VAE latent space for %s dataset (%s)' % (dataset, label) 
                    if label else 'Direct representation of VAE latent space for %s dataset' % dataset)

## Plot 3D PCA for different labels

In [ ]:
for label in labels:
    df = load_vis_data(label)
    data = preload_data_as_tensor(df)
    data_np = evaluate_model(df, data)
    colors = get_colors(df, data_np)
    n_dim = data_np['z'].shape[1]

    if n_dim > 3:
        pca3_data = compute_pca(data_np['z'], n=3)
        plot_data(pca3_data, colors[label], label, plot_components=3,
            hover_data={
                'ids': data_np['ids'], 
                'mode': data_np['ionization_mode_id'], 
                'energy': data_np['collision_energy']},
            title='PCA analysis of VAE latent space for %s dataset (%s)' % (dataset, label) 
                    if label else 'PCA analysis of VAE latent space for %s dataset' % dataset)
    elif n_dim == 3:
        plot_data(data_np['z'], colors[label], label, plot_components=3,
            hover_data={
                'ids': data_np['ids'], 
                'mode': data_np['ionization_mode_id'], 
                'energy': data_np['collision_energy']},
            title='Direct representation of VAE latent space for %s dataset (%s)' % (dataset, label) 
                    if label else 'Direct representation of VAE latent space for %s dataset' % dataset)
